In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT
import glob
import sys
import os
import tqdm

In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:14:1: error: '_OSSwapInt16' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint16_t' (aka 'unsigned short')
uint16_t
^~~~~~~~
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/_OSByteOrder.h:49:1: note: but in 'DarwinFoundation.OSByteOrder' found different return type '__uint16_t' (aka 'unsigned short')
__uint16_t
^~~~~~~~~~
In module 'Darwin':
/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk/usr/include/libkern/arm/OSByteOrder.h:24:1: error: '_OSSwapInt32' has different definitions in different modules; definition in module 'Darwin.libkern.OSByteOrder' first difference is return type is 'uint32_t' (aka 'unsigned int')
uint32_t
^~~~~~~~


Welcome to JupyROOT 6.30/04


In [14]:
# neutron separation energy of Cf-252
Sn = 6.172 # MeV (energy of emitted neutron)

activity = 19858367638938. # Bq/g (total decays, alphas and neutrons) [2]

mCu = 37000000 # one millicurie in Bq

neutron_rate = 2.314e6 # neutrons / sec / ug [1]
neutron_frac = 3.092/100 # fraction of neutrons (remainder alpha decays) [1]

density = 15.1 # g/cm^3 (wikipedia)

Si_density = 2.33 # g/cm^3 (from google)

#typical_purity = mCu/(0.1**3)/density/activity # set so that total is one mCu

typical_purity = 1/4800 # typical fraction of Cf source that has active Cf-252 (numbers from [1] - ish: 5 cm x 1 cm radius block contains ~50 mg of Cf-252)

volume = 0.1**3 # volume (cm^3)
active_mass = density*volume*typical_purity # grams
total_activity = active_mass*activity # Bq (decays per second)

samplerate = neutron_rate*1e6*density*volume*typical_purity # emission rate (neutrons per second)


# [1] https://doi.org/10.1016/S0969-8043(00)00214-1
# [2] https://doi.org/10.1016/j.anucene.2023.109699 
# other: https://www.chemlin.org/isotope/californium-252

def exposure_time(Nprimaries):
    # seconds corresponding to Nprimaries
    return Nprimaries/total_activity

In [15]:
print(total_activity/mCu)

1.688408509842139


In [24]:
def convert_to_one_mCu(rate):
    return rate*mCu/total_activity

convert_to_one_mCu(0.03)

0.01776821179538174

In [16]:
detectors = ['lowmass', 'highmass1', 'highmass2']
def get_rates(*files):
    counts = {det: 0 for det in detectors}
    ncounts = {det: 0 for det in detectors}

    if len(files) < 10:
        tree_iter = lambda N: tqdm.trange(N)
        file_iter = files
    else:
        tree_iter = lambda N: range(N)
        file_iter = tqdm.tqdm(files)
        


    for file in file_iter:
        last_event = -1
        hits = {det: 0 for det in detectors}
        tfile = ROOT.TFile.Open(file, 'READ')
        tree = tfile.Get('tree')
        N = tree.GetEntries()

        for k in tree_iter(N):
            tree.GetEntry(k)
            EventNum = int(getattr(tree, 'EventNum'))
            VolName = getattr(tree, 'VolName')
            ProcName = getattr(tree, 'ProcName')
            hits[VolName] += 1


            if ProcName == 'nCapture':
                ParentVol = getattr(tree, 'ParentVol')

                if (ParentVol == VolName) and (getattr(tree, 'PName') != 'gamma'):
                    ncounts[VolName] += 1

            if abs(EventNum - last_event) > 1e-6: # new event
                last_event = EventNum
                for det in detectors:
                    if hits[det] > 0:
                        counts[det] += 1
                
                hits = {det: 0 for det in detectors}

        tfile.Close()

    return counts, ncounts



In [17]:
Nprimaries = 5e5
T = exposure_time(Nprimaries)
files_15_10 = [
    #'../build/test_data/ratesim_20250109_125611_t0.root',
    '../build/test_data/ratesim_20250109_130755_t0.root',
] # 5.5e6 primaries

files_15_5 = [
    '../build/test_data/ratesim_20250109_133224_t0.root'
] # 5e6 primaries

files_10_5 = [
    '../build/test_data/ratesim_20250109_135033_t0.root'
] # 5e6 primaries

files_10_10 = [
    '../build/test_data/ratesim_20250109_140549_t0.root'
] # 5e6 primaries

files_10_15 = [
    '../build/test_data/ratesim_20250109_142831_t0.root'
] # 5e6 primaries

files_15_15 = [
    '../build/test_data/ratesim_20250109_144110_t0.root'
] # 5e6 primaries

files_20_5 = [
    '../build/test_data/ratesim_20250109_150118_t0.root'
] # 5e6 primaries
# takeaways: not enough poly. For the same distance from the source, way more background events (and a fair amount of neutronElastic/neutronInelastic)

files_15_5_v2 = glob.glob('../build/rate_data_15_5/ratesim_20250111_??????_t?.root')
# 41*5e6 primaries

files_12_5_v2 = glob.glob('../build/rate_data_12_5/ratesim2025011?_??????_t?.root')
# 40*5e6 primaries

files_10_5_v2 = glob.glob('../build/rate_data_10_5/ratesim_20250112_??????_t?.root')
# 40*5e6 primaries

counts, ncounts = get_rates(*files_10_5_v2)
T = exposure_time(40*50e6)

for det in detectors:
    print(f'{det}: {counts[det]/T/1e3:.4g} kHz ({counts[det]:.3g} counts)')
    print(f'    captures: {ncounts[det]/T:.4g} Hz ({ncounts[det]:.3g} counts)')

lowmass: 0 kHz (0 counts)
    captures: 0 Hz (0 counts)
highmass1: 0 kHz (0 counts)
    captures: 0 Hz (0 counts)
highmass2: 0 kHz (0 counts)
    captures: 0 Hz (0 counts)


In [18]:
counts

{'lowmass': 0, 'highmass1': 0, 'highmass2': 0}

In [19]:
ncounts

{'lowmass': 0, 'highmass1': 0, 'highmass2': 0}

In [40]:
np.sqrt(3)/exposure_time(40*50e6)*mCu/total_activity

0.03204293994002423

In [38]:
9.39*mCu/total_activity

5.561450291954484

In [21]:
exposure_time(40*50e6)

32.014796027714844

In [22]:
iontable = 'simdata_20250114_121957_t0.root' # using first versions of throwing ions directly

In [23]:
typical_purity*4000

0.8333333333333334

In [39]:
total_activity/mCu

1.688408509842139